In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow.lite as tflite

# Bild laden
image = cv2.imread("path/to/image.jpg")

# Bildgröße anpassen
image_resized = cv2.resize(image, (128, 128))

# In Graustufen umwandeln
gray = cv2.cvtColor(image_resized, cv2.COLOR_BGR2GRAY)

# Kanten erkennen (für Formanalyse)
edges = cv2.Canny(gray, 50, 150)

cv2.imshow("Edges", edges)
cv2.waitKey(0)
cv2.destroyAllWindows()



# CNN-Modell erstellen
def create_cnn():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax')  # 10 Klassen für Kugelanzahl
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Modell initialisieren
model = create_cnn()
model.summary()






converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Modell speichern
with open("model.tflite", "wb") as f:
    f.write(tflite_model)


    # Modell laden
interpreter = tflite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

# Kamera aktivieren
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_resized = cv2.resize(frame, (128, 128)) / 255.0
    input_data = np.expand_dims(frame_resized, axis=0).astype(np.float32)

    # Modellvorhersage
    interpreter.set_tensor(interpreter.get_input_details()[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(interpreter.get_output_details()[0]['index'])

    print("Erkannte Form:", output_data.argmax())

    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()